In [1]:
import ee 
import geemap

In [2]:
ee.Initialize(project='hotspotstoplight')

RefreshError: ('invalid_grant: Token has been expired or revoked.', {'error': 'invalid_grant', 'error_description': 'Token has been expired or revoked.'})

In [ ]:
aoi = ee.Geometry.Polygon( # all of costa rica
    [[
        [-85.9, 8.0],  # Lower left corner (southwest)
        [-85.9, 11.2], # Upper left corner (northwest)
        [-82.5, 11.2], # Upper right corner (northeast)
        [-82.5, 8.0],  # Lower right corner (southeast)
        [-85.9, 8.0]   # Closing the polygon by repeating the first point
    ]]
)

In [ ]:
// Define your area of interest
var aoi = ee.Geometry.Point([longitude, latitude]); // Replace with your area of interest coordinates

// Import the Landsat 8 collection and filter by date and region
var collection = ee.ImageCollection('LANDSAT/LC08/C01/T1')
                  .filterDate('2020-01-01', '2020-12-31') // Adjust dates accordingly
                  .filterBounds(aoi);

// Calculate NDVI
var addNDVI = function(image) {
  var ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI');
  return image.addBands(ndvi);
};

// Calculate NDBI
var addNDBI = function(image) {
  var ndbi = image.normalizedDifference(['B6', 'B5']).rename('NDBI');
  return image.addBands(ndbi);
};

// Calculate LST with emissivity correction
// Placeholder function: Adjust emissivity calculation based on your needs
var calculateLST = function(image) {
  // Add LST calculation logic here based on the bands available in your image
  var lst = image.select('B10').subtract(273.15).rename('LST'); // Simplified example
  return image.addBands(lst);
};

// Add longitude and latitude bands
var addLonLat = function(image) {
  var ll = ee.Image.pixelLonLat().select(['longitude', 'latitude']);
  return image.addBands(ll);
};

// Apply the functions to the collection
var processedCollection = collection.map(addNDVI)
                                     .map(addNDBI)
                                     .map(calculateLST)
                                     .map(addLonLat);

// Example of stacking the layers to create a raster stack (Not executable in GEE directly, conceptual)
var stackLayers = function(image) {
  var stack = image.select('NDVI').addBands(image.select('NDBI'))
                   .addBands(image.select('LST'))
                   .addBands(image.select('longitude', 'latitude'));
  return stack;
};

var rasterStack = processedCollection.map(stackLayers);

// Random Forest Model Training
// Define the features and label
var features = rasterStack.select(['NDVI', 'NDBI', 'LST', 'longitude', 'latitude']);
var label = 'LST'; // Example label, adjust based on your use case

// Split the data into training and testing (this is a conceptual step)
var training = features.sample({
  region: aoi,
  scale: 30,
  numPixels: 5000
});

var trainedModel = ee.Classifier.smileRandomForest(10) // Number of trees
                    .train({
                      features: training,
                      classProperty: label,
                      inputProperties: ['NDVI', 'NDBI', 'emissivity', 'longitude', 'latitude']
                    });

// Apply the trained model to a dataset (conceptual, adjust according to your needs)
var classified = features.classify(trainedModel);

// Display or Export the results
// Map.addLayer(classified, {min: 0, max: 1}, 'Classified Image');
// Export.image.toDrive({
//   image: classified,
//   description: 'classified_image',
//   scale: 30,
//   region: aoi
// });

